## This notebook loads EPH data, cleans it and arranges it to be used as 'training sets'. 
That is, for fitting any Machine Learning model.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

import numpy as np
pd.options.display.max_columns = 99

from IPython.display import display, HTML

import glob

In [2]:
startyr = 2020
endyr = 2021

In [3]:
radio_ref = pd.read_csv('./../data/info/radio_ref.csv')
# radio_ref[['PROV','NOMPROV','DPTO', 'NOMDPTO']].drop_duplicates().to_csv('./../data/DPTO_PROV.csv', index = False)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)
AGLO_Region = radio_ref[['AGLOMERADO', 'Region']].drop_duplicates()

# Decision sobre cual es la region de un aglomerado. GBA tiene que ir a Gran Buenos Aires, aunque algunos de sus radios en partidos como Rodriguez, Escobar, etc sean region pampeana.
# Viedma Patagones, se tendria que tirar de un lado, y la mayoria de sus radios, son Patagonia.
# Se tiene que corregir a mano, porque el AGLO 0 SI tiene varias regiones.

AGLO_Region = AGLO_Region.loc[~((AGLO_Region.AGLOMERADO == 33) & (AGLO_Region.Region == 'Pampeana'))]
AGLO_Region = AGLO_Region.loc[~((AGLO_Region.AGLOMERADO == 93) & (AGLO_Region.Region == 'Pampeana'))]

### Match column names

names_censo = ['IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO',
    'V01', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14', 'H13',
      'P07', 'P08', 'P09', 'P10', 'P05']


names_EPH = ['IX_TOT','CH04','CH06','CONDACT', 'AGLOMERADO',
    'IV1', 'IV3', 'IV4','IV5','IV6','IV7','IV8','IV10','IV11','II1','II2','II7','II8','II9',
    'CH09','CH10','CH12','CH13','CH15']

col_mon = [u'P21', u'P47T', u'PP08D1', u'TOT_P12', u'T_VI', u'V12_M', u'V2_M', u'V3_M', u'V5_M']



In [4]:
cpi = pd.read_csv('./../data/info/indice_precios_Q.csv', index_col=0)

cpi.index = pd.to_datetime(cpi.index)
cpi = cpi['2003':]

In [5]:
cpi_M = pd.read_csv('./../data/info/indice_precios_M.csv', index_col=0)
cpi_mes_actual = cpi_M.iloc[-1][0]

cpi_M.tail(2)

,index
2020-12-31,3188.361836
2021-01-31,3270.799721


In [6]:
cpi_mes_actual

3270.7997208542934

In [7]:
from pandas.tseries.offsets import MonthEnd

## Cargar EPHs

Los microdatos de la Encuesta Permanente de Hogares se pueden descargar con:

``git clone https://github.com/matuteiglesias/microdatos-EPH-INDEC.git``

(darle star al repositorio)

tomando pull del mismo repositorio se va a poder actualizar con los nuevos microdatos a medida que se publican. Siempre y cuando el repositorio se mantenga actualizado. Y hasta que no se supere la capacidad de almacenamiento en repo github.

In [9]:
import os

path ='./../../microdatos-EPH-INDEC/microdatos/' # depende de donde hayamos descargado los microdatos
# path ='./../../EPH/microdatos/' # depende de donde hayamos descargado los microdatos

for y in range(startyr, endyr):
    print(y)
    yr = str(y)[2:]
    allFiles = glob.glob(path + 'hogar/*'+str(yr)+'.txt')
    frame = pd.DataFrame()
    list_ = []
    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0, delimiter = ';',
                        usecols = ['CODUSU','ANO4','TRIMESTRE','IX_TOT', 'AGLOMERADO',
        'IV1', 'IV3', 'IV4','IV5','IV6','IV7','IV8','IV10','IV11','II1','II2','II7','II8','II9']) 
        ['II2', 'IV5', 'IX_TOT', 'II7', 'IV4', 'II1', 'IV7', 'IV6', 'IV11', 'IV8', 'IV3', 'II8', 'IV1', 'IV10', 'II9']
        
        print(len(df))
        list_ += [df]
    df = pd.concat(list_)

    # Correcciones Respuestas. Para que matchee censo
    df = df.loc[df.IV1 != 9]
    df['IV10'] = df['IV10'].map({1: 1, 2: 2, 3: 2, 0: 0, 9: 9})
    df['II9'] = df['II9'].map({1: 1, 2: 2, 3: 2, 4: 4, 0: 0})
    df['II7'] = df['II7'].map({1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 6, 8: 6, 9: 6, 0: 0})
    df['II9'] = df['II9'].map({1: 1, 2: 2, 3: 2, 4: 4, 0: 0})
    df['IX_TOT'] = df['IX_TOT'].clip(0, 8)
    
    hogar = df
    hogar = hogar.drop_duplicates()
    print(hogar.shape)

    allFiles = glob.glob(path + 'individual/usu_individual*'+str(yr)+'.txt')
    frame = pd.DataFrame()
    list_ = []
    for file_ in allFiles:
        print(file_)
    #     print(file_)
        df = pd.read_csv(file_,index_col=None, header=0, delimiter = ';',
                         usecols = ['CODUSU','ANO4','TRIMESTRE','CH04','CH06', 'AGLOMERADO', 'CH09','CH10','CH12','CH13','CH15'] +\
                         ['CH07', 'ESTADO','CAT_INAC','CAT_OCUP','PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59', 'PP07H', 'PP07I', 'PP07J', 'PP07K',
                         'P47T', 'V3_M', 'T_VI', 'V12_M', 'TOT_P12', 'V5_M','V2_M', 'PP08D1', 'P21'])
        df = df.rename(columns = {'ESTADO': 'CONDACT'})

#         display(df.head())
# revisar estado, condact, cat ocup, cat inac.
    # For the regression training set. But for these the ANO4 TRIMESTRE is important.. Also we need more memory.
    #                      ['P21','P47T',,'CH08','CH16','TOT_P12','T_VI','V10_M','V11_M','V12_M','V18_M','V19_AM','V21_M','V2_M','V3_M',
    #             'V4_M','V5_M','V8_M','V9_M','PP08D1','PP08D4','PP08F1','PP08F2','PP08J1','PP08J2','PP08J3','PP10A','PP10C','PP10D','PP10E'])
#         print(len(df))
        list_ += [df]
    df = pd.concat(list_)

    # Correcciones Respuestas. Para que matchee censo
    df['CH15'] = df['CH15'].map({1:1, 2:1, 3:1, 4:2, 5:2, 9:0})
    df['CH06'] = df['CH06'].clip(0)
    df['CH09'] = df['CH09'].map({1:1, 2:2, 0:2, 3:2})
    df.loc[df['CH06'] < 14, 'CONDACT'] = 0 # Menores de 14 van con CONDACT 0, como en el Censo
    
    ## En Censo, Jardin y educacion especial no preguntan terminado si/no.
    df['CH12'] = df.CH12.replace(99, 0)
    df.loc[df.CH12.isin([0, 1, 9]), 'CH13'] = 0

#     df['MAYOR'] = df['CH06'] >= 14 
#     df['MAYOR'] = df['CH06'] // 7
#     df['CONDACT'] = df['CAT_OCUP'].fillna(-1)

    indiv = df
    indiv = indiv.dropna(subset = ['P47T'])
    print(indiv.shape)

    indiv_table = indiv[list(indiv.columns.difference(hogar.columns)) + ['CODUSU', 'ANO4', 'TRIMESTRE', 'AGLOMERADO']]
    EPH = hogar.merge(indiv_table, on = ['CODUSU', 'ANO4', 'TRIMESTRE', 'AGLOMERADO'], indicator = True)

    print('Hogar - Indiv merged:')
    print(EPH.shape)

    
#     EPH = EPH.loc[EPH.P47T != -9]
    
    EPH = EPH.merge(AGLO_Region)

    EPH_no_aglo = EPH.copy(); 
    EPH_no_aglo['AGLOMERADO'] = 0

    EPH = pd.concat([EPH, EPH_no_aglo]).reset_index(drop = True)

    print('No aglo agregado:')
    print(EPH.shape)
    
    # Quarters / deflation
    EPH['Q'] = EPH.ANO4.astype(str) + ':' + (3*EPH.TRIMESTRE).astype(str)
    EPH['Q'] = pd.to_datetime(EPH['Q'], format='%Y:%m') + MonthEnd(1)
#     cpi_ultimo_Q = indice_precios['index'].values[-1]
    
    EPH[col_mon] = cpi_mes_actual*EPH[col_mon].div(EPH[['Q'] + col_mon].merge(cpi, on = 'Q', how = 'left')['index'].values, 0)
    
    # 2018Q3 -> Mar19 1.3156
    # 2018Q3 -> Abr19 1.361
#     EPH[col_mon] = 1.361*EPH[col_mon]
    
    EPH[col_mon] = EPH[col_mon].round()
    
    print('deflactado:')
    print(EPH.shape)
    display(EPH[col_mon].mean())
    
    
    training = EPH.rename(columns = dict(zip(names_EPH, names_censo)))
    
    if not os.path.exists('./../data/training/'):
        os.makedirs('./../data/training/')
    
    training.to_csv('./../data/training/EPHARG_train_'+str(yr)+'.csv', index = False)

2020
16845
11976
(28799, 19)
./../../microdatos-EPH-INDEC/microdatos/individual/usu_individual_t120.txt
./../../microdatos-EPH-INDEC/microdatos/individual/usu_individual_t220.txt
(88613, 33)
Hogar - Indiv merged:
(90788, 49)
No aglo agregado:
(181576, 50)
deflactado:
(181576, 51)


P21        11228.098581
P47T       19412.007071
PP08D1     23814.104135
TOT_P12      783.981132
T_VI        6828.807386
V12_M        374.599330
V2_M        4507.146583
V3_M         184.668315
V5_M         732.604023
dtype: float64

### Ranking de AGLOS y Regiones

In [10]:
df_list = []
# for yr in [str(s) for s in [2006, 2011, 2016]]:
# for yr in [str(s) for s in [2015, 2020]]:
for yr in [str(s) for s in [2020]]:  # Esto depende de los anios en que tengamos la microdata, podemos elegir
# for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    train = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')
    train = train.loc[train.P47T >= -0.001].fillna(0)#.sample(400000)
    df_list += [train]
    
train_df = pd.concat(df_list)

AGLO_rk = train_df.loc[train_df.CAT_OCUP == 3].groupby(['AGLOMERADO'])[['P47T']].mean().sort_values('P47T').reset_index().reset_index().rename(columns = {'index':'AGLO_rk'})
Reg_rk = train_df.loc[train_df.CAT_OCUP == 3].groupby(['Region'])[['P47T']].mean().sort_values('P47T').reset_index().reset_index().rename(columns = {'index':'Reg_rk'})

AGLO_rk['AGLO_rk'] = AGLO_rk.AGLO_rk/AGLO_rk.AGLO_rk.max()
AGLO_rk.to_csv('./../data/info/AGLO_rk', index = False)
Reg_rk['Reg_rk'] = Reg_rk.Reg_rk/Reg_rk.Reg_rk.max()
Reg_rk.to_csv('./../data/info/Reg_rk', index = False)

# check it out
# AGLO_rk.merge(pd.read_csv('./../data/info/aglo_labels.csv'))

2020


## Guardar training dataset

In [11]:
AGLO_rk = pd.read_csv('./../data/info/AGLO_rk')
Reg_rk = pd.read_csv('./../data/info/Reg_rk')

df_list = []
for yr in [str(s) for s in range(startyr, endyr)]:
# for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    train = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')#.drop(['AGLO_rk', 'Reg_rk'], axis = 1)
    train = train.loc[train.P47T >= -0.001].fillna(0)
    train = train.merge(AGLO_rk[['AGLOMERADO', 'AGLO_rk']]).merge(Reg_rk[['Region', 'Reg_rk']])
    train.to_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv', index = False)

2020


## Listo. Salvado el training set.

### Borradores sobre los nombres de columnas..

In [ ]:
# # Misma info, distinto nombre. 
# # Censo INDEC 
# md_1 = table[['IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO', #las que no se erran, cant pers, sexo, edad, act, aglo
#     'V01', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14', 'H13',
#       'P07', 'P08', 'P09', 'P10', 'P05']] #las x que buscan matches un poquito mas laxamente


# #Mismas cosas, distinto nombre de columna para
# # EPH INDEC
# md_2 = EPH[['IX_TOT','CH04','CH06','CONDACT', 'AGLOMERADO',
#     'IV1', 'IV3', 'IV4','IV5','IV6','IV7','IV8','IV10','IV11','II1','II2','II7','II8','II9',
#     'CH09','CH10','CH12','CH13','CH15']]

# # # Controlar superposicion de columnas

# # for i in range(len(md_1.columns))[:2]: 
# #     print('\n')
# #     for md in [md_1, md_2]:
# #         col = md.columns[i]
# #         print(col)
# #         print(md[col].value_counts().sort_index()/len(md))

# md_2.columns = md_1.columns

In [ ]:
# import numpy as np
# # Agregar $$$. En millones de usd
# # En millones de usd (USD = 30 ARS)
# _USD = 30.5 #ARS
# np.round(res_1.sum()/_USD/1e6, 1).sort_values().tail(14)

# #PPALES
# # negocio que no trabajo no laborable (V9_M)
# # alquiler no laborable (V8_M)
# # indemnizacion despido no laborable (V3_M)
# # comision Ocupacion ppal (PP08F1)
# # cuota alimentos no laborable (V12_M)
# # subsidio ayuda social no laborable (V5_M)
# # TOTAL otras ocupacions(TOT_P12)
# # jubilacion no laborable (V2_M)
# # TOTAL no laborables (T_VI)
# # sueldo Ocupacion ppal(PP08D1)
# # TOTAL Ocupacion ppal (P21)
# # TOTAL TOTAL (P47T)

In [ ]:
# # Desoc, NA = 0. Not good.
# variables = ['PP07J', #turno habitual
#  'PP10D', #Desoc. Ha trabajado alguna vez?
#  'PP10C', #Desoc. Hizo changa mientras buscaba?
#  'PP07K', # Oc. ppal. Inc. serv. dom. Cobra con recibo
#  'PP07G2', # Oc. ppal. Inc. serv. dom. aguinaldo
#  'PP07G_59', # Oc. ppal. Inc. serv. dom. ninguno
#  'PP07G3', # Oc. ppal. Inc. serv. dom. dias enfermedad
#  'PP10E', # Desoc. Tiempo de que termino su ultimo trabajo/changa
#  'PP07H', # Oc. ppal. Inc. serv. dom. descuento jubilatorio
#  'PP07G4', # Oc. ppal. Inc. serv. dom. obra social
#  'PP07I', # Oc. ppal. Inc. serv. dom. Aporta jub por sí mismo 
#  'PP07G1', # Oc. ppal. Inc. serv. dom. vacaciones pagas
#  'CH07'] #Est civil

# #en miles de USD
# # res_byDPTO = res_DPTO.groupby(['DPTO', 'NOMDPTO', 'NOMPROV'])[variables].mean()
# res_byDPTO = res_DPTO.groupby(['RADIO_REF_ID'])[variables].mean()


# s = np.round(res_byDPTO, 2).sort_values(by = 'PP07K')#.head() 
# s.style.bar(color='#d65f5f')
# #  'CAT_OCUP', #CAT_INAC
# #  'CAT_INAC', #CAT_INAC

# #  'CH08', obra social/salud. nums altos como para mean
# # 'CH16', # Donde vivia hace 5. Desconfiable


In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

# for col in result.columns:
#     print('\n')
#     print(col)
#     df_ = result.loc[result.P03 > 2]
#     print(df_[col].value_counts().sort_index())